# Catchment-scale forecasts of veg condition

Using the pre-computed NDVI quarterly climatologies (these have been calculated over all of NSW, but only a subset from the Gwdir catchment is stored on the Sandbox), see if we can predict NDVI one-month in advance.

- Rescaling datasets to 210x210m to speed up testing. 

TODO:
- Run seasonal anomaly using `seasonal_anomalies.ipynb` and compare to anomaly calculation in this script. This will provide a validation of the code here

In [ ]:
# !pip install xarray --upgrade

In [ ]:
from datacube import Datacube
import numpy as np
import geopandas as gpd
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import lag_plot
from pandas.plotting import autocorrelation_plot
from datacube.utils import geometry
from scipy import stats, signal
from datacube.utils.geometry import assign_crs
from statsmodels.tsa.ar_model import AutoReg
from odc.algo import xr_reproject
import sys
import os
from datacube.utils.cog import write_cog

sys.path.append('../dea-notebooks/Scripts')
from dea_datahandling import load_ard
from dea_plotting import map_shapefile
from dea_dask import create_local_dask_cluster
from dea_temporal_statistics import fast_completion, smooth
from dea_spatialtools import xr_rasterize

In [ ]:
create_local_dask_cluster()

## Analysis Parameters

In [ ]:
mean_tifs = 'data/climatologies/mean/'
std_tifs = 'data/climatologies/std/'

shp = 'data/mdb_shps/GWYDIR RIVER.shp'
time = ('2014-01', '2018-12')

In [ ]:
gdf = gpd.read_file(shp).to_crs('EPSG:4326')
map_shapefile(gdf, attribute='BNAME')

## Load data

In [ ]:
geom = geometry.Geometry(
        gdf.geometry.values[0].__geo_interface__, geometry.CRS(
            'epsg:4326'))

dc = Datacube(app='whatevrr')

query = {"geopolygon": geom,
         'time': time,
         'measurements':['nbart_red', 'nbart_nir'],
         'output_crs' :'EPSG:3577',
         'resolution' : (-120, 120),
         'resampling' :{"fmask": "nearest", "*": "bilinear"}}

In [ ]:
ds = load_ard(dc=dc, 
              dask_chunks={'x':750, 'y':750, 'time':-1},
              products=["ga_ls8c_ard_3"],
              group_by="solar_day",
              **query)

mask = xr_rasterize(gdf.iloc[[0]], ds)

ds = ds.where(mask).astype('float32')

## Calculate NDVI

In [ ]:
ds = (ds.nbart_nir - ds.nbart_red) / (ds.nbart_nir + ds.nbart_red)

## Fill gaps and resample to monthly

TODO: switch to `fast_completion` (wrap function inside xr.map_blocks or xr.apply_ufunc)

In [ ]:
ndvi_monthly = ds.interpolate_na(dim='time', method='linear').resample(time='1M').mean()#.rolling(time=3, min_periods=3, center=True).mean()

## Load climatologies
Data is a subset over the Gwydir catchment

In [ ]:
list_of_mean_tifs = os.listdir(mean_tifs)
list_of_std_tifs = os.listdir(std_tifs)

chunks = {'x':1000, 'y':1000}
# quarterly NDVI mean climatologies
x = []
for tif in list_of_mean_tifs:
    y = assign_crs(xr.open_rasterio(mean_tifs+tif, chunks=chunks))
    y = xr_reproject(y,ds.geobox,"bilinear")
    x.append(y.rename(tif[15:-11]))
    
clim_mean = xr.merge(x).squeeze().drop('band')

# quarterly NDVI std. dev. climatologies
x = []
for tif in list_of_std_tifs:
    y = assign_crs(xr.open_rasterio(std_tifs+tif, chunks=chunks))
    y = xr_reproject(y,ds.geobox,"bilinear")
    x.append(y.rename(tif[14:-11]))
    
clim_std = xr.merge(x).squeeze().drop('band')

## Calculate standardized anomalies

Loop through each year+quarter and substract climatology, then rebuild time-series

In [ ]:
# First compute our arrays
clim_std = clim_std.compute()
clim_mean = clim_mean.compute()
ndvi_monthly = ndvi_monthly.compute()

In [ ]:
clim_mean.MAM.plot(size=5)

#### TODO: BETTER HANDLE YEARS LOOP...KEEPS GOING THROUGH ENTIRE YEAR EVEN IF THERE'S ONLY A FEW MONTHS FROM THAT YEAR

In [ ]:
import warnings
warnings.filterwarnings("ignore")

#define the 3-month intervals
quarter= {'JFM': [1,2,3],
           'FMA': [2,3,4],
           'MAM': [3,4,5],
           'AMJ': [4,5,6],
           'MJJ': [5,6,7],
           'JJA': [6,7,8],
           'JAS': [7,8,9],
           'ASO': [8,9,10],
           'SON': [9,10,11],
           'OND': [10,11,12],
           'NDJ': [11,12,1],
           'DJF': [12,1,2],
                      }
#get the unique years in ds
years = [str(i) for i in np.unique(ndvi_monthly.time.dt.year.values)]

#loop through each 3 month period and calculate the anomaly
z=[]
for year in years:
    for q in quarter:
        months = quarter.get(q)
        if (q == "DJF") or (q == "NDJ"):
            time=(year + "-" + str(months[0]), str(int(year) + 1) + "-" + str(months[2]))
        else:
            time = (year + "-" + str(months[0]), year + "-" + str(months[2]))
        obs=ndvi_monthly.sel(time=slice(time[0], time[1])).mean('time')
        m=clim_mean[q]
        s=clim_std[q]
        anom = (obs - m) / s
        print(year+'_'+q)
        anom.rename(year+'_'+q)
        z.append(anom)

#### TODO: auto handle dates

In [ ]:
# Build back into time-series
stand_anomalies=xr.concat(z, dim=pd.date_range(start='2/2014', end='2/2019', freq='M')).rename({'concat_dim':'time'})

stand_anomalies.mean(['x','y']).plot(figsize=(11,5))

## Make a forecast

`AutoReg` doesn't like the all-NaN's slices outide the mask extent, run `stand_anomalies.fillna(-999)`

In [ ]:
mask = stand_anomalies.notnull().all('time')

In [ ]:
#mask where its all-NaN's
stand_anomalies = stand_anomalies.fillna(-999)

In [ ]:
test_length=1
window=20
lags=20

In [ ]:
%%time
def xr_autoregress(da, test_length, window, lags):
    #dropna conveneiently with pandas
    da =  da[~np.isnan(da)]
    # split dataset
    train, test = da[1:len(da)-test_length], da[len(da)-test_length:]
    # train autoregression
    model = AutoReg(train, lags=lags)
    model_fit = model.fit()
    coef = model_fit.params

    # walk forward over time steps in test
    history = train[len(train)-window:]
    history = [history[i] for i in range(len(history))]

    predictions = list()
    for t in range(len(test)):
        length = len(history)
        lag = [history[i] for i in range(length-window,length)]
        yhat = coef[0]
        for d in range(window):
            yhat += coef[d+1] * lag[window-d-1]
        obs = test[t]
        predictions.append(yhat)
        history.append(obs) 
    
    return np.array(predictions).flatten()

predict = xr.apply_ufunc(xr_autoregress,
                      stand_anomalies, #.chunk(dict(x=750,y=750,time=-1)),
                      kwargs={'test_length':test_length,'window':window,'lags':window},
                      input_core_dims=[['time']],
                      output_core_dims=[['predictions']], 
                      output_sizes=({'predictions':test_length}),
                      exclude_dims=set(('time',)),
                      vectorize=True,
                      dask="parallelized",
                      output_dtypes=[stand_anomalies.dtype]).compute()

print(predict)

In [ ]:
predict = predict.where(mask)

In [ ]:
predict.plot(size=6, vmin=-2.0, vmax=2, cmap='BrBG')
plt.title('Standardised NDVI Anomaly one-month prediction');

In [ ]:
stand_anomalies=stand_anomalies.where(mask)
stand_anomalies.isel(time=-1).plot(size=6, vmin=-2.0, vmax=2, cmap='BrBG')
plt.title('Standardised NDVI Anomaly observation');

In [ ]:
diff = predict - stand_anomalies.isel(time=-1)

diff.plot(size=6, vmin=-2.0, vmax=2, cmap='RdBu')
plt.title('Difference');

In [ ]:
diff.mean(['x','y'])

### Can we speed this up by manually looping through pixels in cython?

    from cython.parallel import prange, parallel, threadid
    def predict_yhat(floating [:, :, :, :] da, test_length, window, lags, [:,:,:] result, num_threads):    
    da, test_length, window, lags
        with nogil, parallel(num_threads=number_of_threads):
            for row in prange(m, schedule='static'):
                for col in range(q):
                        # do the prediction
                        da =  da[~np.isnan(da)]
                        # split dataset
                        train, test = da[1:len(da)-test_length], da[len(da)-test_length:]
                        # train autoregression
                        model = AutoReg(train, lags=lags)
                        model_fit = model.fit()
                        coef = model_fit.params

                        # walk forward over time steps in test
                        history = train[len(train)-window:]
                        history = [history[i] for i in range(len(history))]

                        predictions = list()
                        for t in range(len(test)):
                            length = len(history)
                            lag = [history[i] for i in range(length-window,length)]
                            yhat = coef[0]
                            for d in range(window):
                                yhat += coef[d+1] * lag[window-d-1]
                            obs = test[t]
                            predictions.append(yhat)
                            history.append(obs) 

                        return[row, col] = np.array(predictions).flatten()
